In [2]:
#pip install PySimpleGUI

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import PySimpleGUI as sg
import os

In [4]:
working_directory=os.getcwd()
layout=[
    [sg.Text("Upload the Item Master Data:")],
    [sg.InputText(key="-FILE_PATH1-"),
    sg.FileBrowse(initial_folder=working_directory,file_types=[("Excel Files","*.xlsx")])],
    [sg.Text("Upload the Route Data:")],
    [sg.InputText(key="-FILE_PATH2-"),
    sg.FileBrowse(initial_folder=working_directory,file_types=[("Excel Files","*.xlsx")])],
    [sg.Text("Upload the Order Data:")],
    [sg.InputText(key="-FILE_PATH3-"),
    sg.FileBrowse(initial_folder=working_directory,file_types=[("Excel Files","*.xlsx")])],
    [sg.Button("Submit"),sg.Exit()]
]

window=sg.Window("Loading Prediction",layout)

while True:
    event, values=window.read()
    if event in (sg.WIN_CLOSED,"Exit"):
        break
    elif event=="Submit":
        excel_file1=values["-FILE_PATH1-"]
        excel_file2=values["-FILE_PATH2-"]
        excel_file3=values["-FILE_PATH3-"]
        df=pd.read_excel(excel_file1)
        df1=pd.read_excel(excel_file2)
        df2=pd.read_excel(excel_file3)
        data = {"type": [18,22,26,29],"length": [215.13,262.63,310.63,346.63],"height": [97.12,97.12,97.12,97.12],"width": [92.13,97.16,97.16,97.16],"weight": [8950,8950,30900,30900]}
        truck=pd.DataFrame(data)#set truck types
        truck["area"]=truck["length"]*truck["width"]
        df1_1=df1[(df1["GeoStop"]==0)|(df1["CustomerOrder"].notna())].reset_index() #data cleaning
        #set No. for each route
        num=0
        df1_1["RouteNo"] = np.nan
        for i in range(len(df1_1)):
            if df1_1.loc[i,"GeoStop"]==0:
                df1_1.loc[i,"RouteNo"]="na"
                num+=1
            else:
                df1_1.loc[i,"RouteNo"]=num
        df1_2=df1_1[df1_1["RouteNo"]!="na"].reset_index() #get clean route data
        df3_1=pd.merge(df1_2,df2, how='left', left_on=["CustomerOrder"], right_on=["SO#"])#link route data with order data
        df3_2=df3_1[df3_1["Product Line"].isin(['LSC', 'PNL', 'PT', 'MD', 'NS', 'SP', 'LVT'])]#remove SL,QZ
        df3=df3_2[["RouteNo","RouteName","Customer#","OrderWeight","Weight","Item#","Product Line","Qty","SO#"]]
        #link order to IMD to get crates dimensions
        df4=pd.merge(df3,df, how='left', left_on=["Item#"], right_on=["ItemNumber"])
        #standardize crates' length and width
        df4["length"]=np.where(df4["CrateSizeLength"]>=df4["CrateSizeWidth"],df4["CrateSizeLength"],df4["CrateSizeWidth"])
        df4["width"]=np.where(df4["CrateSizeLength"]>=df4["CrateSizeWidth"],df4["CrateSizeWidth"],df4["CrateSizeLength"])
        df4["L"]=np.where((df4["length"]<=32)&(df4["width"]<=32),32,np.where((df4["length"]<=48)&(df4["width"]<=40),48,df4["length"])) #fit items less than 48*40/32*32 into standard crate size
        df4["W"]=np.where((df4["length"]<=32)&(df4["width"]<=32),32,np.where((df4["length"]<=48)&(df4["width"]<=40),40,df4["width"])) #fit items less than 48*40/32*32 into standard crate size
        #choose columns
        df5=df4[["length","width","Product Line_x","Customer#","RouteNo","RouteName","SO#","Item#","Qty","PcsPerCrate","Weight","WtPerCrate","CrateSizeHeight","L","W"]]
        #calculate crate number, partial crate percentage, max stackable crates number, crates number for occupying space, full crates number and partial crates for each item
        df5["cratenum"]=round(df5["Qty"]/df5["PcsPerCrate"],4)
        df5["leftpercent"]=round((df5["Qty"]%df5["PcsPerCrate"])/df5["PcsPerCrate"],4)
        df5["maxstack"]=np.where((df5["Product Line_x"]=="LVT")|(df5["Product Line_x"]=="PT"),np.where((df5["CrateSizeHeight"]<=48) & (3000/df5["WtPerCrate"]>1),np.where(97.12/df5["CrateSizeHeight"]>3000/df5["WtPerCrate"],3000/df5["WtPerCrate"],97.12/df5["CrateSizeHeight"]),1),1)
        df5["positnum"]=np.where(df5["maxstack"]>=2,np.ceil(df5["cratenum"]/df5["maxstack"]),np.where((df5["maxstack"]>1)&(df5["leftpercent"]/np.floor(df5["cratenum"])<=(df5["maxstack"]-np.floor(df5["maxstack"]))),np.floor(df5["cratenum"]),np.ceil(df5["cratenum"])))
        df5["cratearea"]=df5["L"]*df5["W"]
        df5["full"]=np.floor(df5["cratenum"])
        df5["part"]=round(df5["cratenum"]-df5["full"],4)
        #drop unmapped order records
        df5_1=df5[df5["SO#"].notna()==True]
        #sort by route no, customer# and dimensions
        df6=df5_1.sort_values(by=['RouteNo','Customer#','L','W'],ascending= False).reset_index()
        #get partial order items and calculate each order's packaged crates number
        df7=df6[df6["cratenum"]<1].groupby(["SO#"])["cratenum"].aggregate(["count","sum"]).reset_index()
        so=[]
        packper=[]
        for i in range(len(df7)):
            so.append(df7.loc[i,"SO#"]) 
            packper.append(df7.loc[i,"sum"]) #total packaged number
        #package partial items into the crate of largest size for each same order
        df8=df6[df6["cratenum"]<1].sort_values(by=['SO#','L','W'],ascending= False).reset_index()
        df8["packcrate"]=np.nan
        for i in range(len(so)):
            n=0
            for j in range(len(df8)):
                if so[i]==df8.loc[j,"SO#"]:
                    if n==0:
                        df8.loc[j,"packcrate"]=packper[i]
                        n=n+1
                    else:
                        df8.loc[j,"packcrate"]=0
                else:
                    continue
        #mapping the calculated packaged info into the route orders
        for i in range(len(df6)):
            for j in range(len(df8)):
                if (df8.loc[j,"SO#"]==df6.loc[i,"SO#"]) & (df8.loc[j,"Item#"]==df6.loc[i,"Item#"]):
                    df6.loc[i,"part"]=df8.loc[j,"packcrate"]
                    df6.loc[i,"positnum"]=np.ceil(df8.loc[j,"packcrate"])
                else:
                    continue
        routeno=[]
        routename=[]
        routeinfo=[]
        routeresult=[] #truck type suggestion
        fullnum=[] #full crates number
        partnum=[] #partial crates number
        positnum=[] #space occupied number
        unloadedfull=[] #unloaded full crates number
        unloadedpart=[] #unloaded partial crates number
        leftwt=[] #save left weight capacity for each truck type
        leftarea=[] 

        #decide which truck to predict by first checking the route orders' total weight
        for route in range(1,df6["RouteNo"].max()+1):
            df9=df6[df6["RouteNo"]==route].sort_values(by=['Customer#','L','W'],ascending= False).reset_index()
            routeno.append(route)
            if len(df9.RouteName.unique())==0:
                routename.append("")
            else:
                routename.append(df9.RouteName.unique()[0])
            routeinfo.append("Route item records:"+str(len(df9))+".Total weight of order's in this route:"+str(round(df9['Weight'].sum(),4)))

            types=[] #save truck type
            loadrec=[] #save loaded number for each truck type


            for k in range(4):
                l=truck.loc[k,'length']
                w=truck.loc[k,'width']
                wt=truck.loc[k,'weight']
                area=truck.loc[k,'area']

                x=[]  #for finding the largest length for each row
                row=1   #the row of crate
                df9["load"] = np.nan
                for i in range(len(df9)):
                    if (df9.loc[i, 'Weight']<wt): #check weight constraint
                        if(df9.loc[i, 'positnum']==0): #packaged with other items, do not occupy space separately
                            df9.loc[i, 'load']="Yes" #marked as loaded
                            wt-=df9.loc[i, 'Weight'] #deduct left truck weight
                        else:
                            if (df9.loc[i, 'L']<l): #check length constraint
                                for j in range(int(df9.loc[i, 'positnum'])): #for each item that occupy 1 or more positions, check each position's constraints
                                    if(df9.loc[i, 'W']<w): #check weight constraint
                                        w-=df9.loc[i, 'W'] #deduct left truck width                           
                                        x.append(df9.loc[i, 'L']) #record crates' length in the same row
                                        if j==int(df9.loc[i, 'positnum']-1): #check whether all the crates of the same item record have been loaded
                                            df9.loc[i, 'load']="Yes" #marked as loaded
                                            wt-=df9.loc[i, 'Weight'] #deduct left truck weight
                                            area-=df9.loc[i, 'positnum']*df9.loc[i, 'cratearea'] #deduct left truck area
                                        else:
                                            continue
                                    else:
                                        l-=max(x) #deduct left truck length by the largest crate's length
                                        row+=1 #move to next row
                                        w=truck.loc[k,'width'] #reset the left truck width
                                        x=[] #release the record of crates' length in the same row
                                        if (df9.loc[i, 'L']<l):
                                            if(df9.loc[i, 'W']<w):
                                                w-=df9.loc[i, 'W']                            
                                                x.append(df9.loc[i, 'L'])
                                                if j==int(df9.loc[i, 'positnum']-1):
                                                    df9.loc[i, 'load']="Yes"
                                                    wt-=df9.loc[i, 'Weight']
                                                    area-=df9.loc[i, 'positnum']*df9.loc[i, 'cratearea']
                                                else:
                                                    continue
                            else:
                                break
                    else:
                        break
                types.append(truck.loc[k,'type'])
                loadrec.append(len(df9[df9.load=="Yes"]))


            result =pd.DataFrame({"type": types,"loadrecord": loadrec}) 

            best=29
            if len(df9)==0:
                routeresult.append("No records.")
            elif result["loadrecord"].max()!=len(df9):
                routeresult.append("Cannot load all the items.")
            else:
                best=result[result["loadrecord"]==len(df9)].type.min()
                routeresult.append("Best truck type:"+str(best))

            l=truck[truck["type"]==best].length.values[0]
            w=truck[truck["type"]==best].width.values[0]
            wt=truck[truck["type"]==best].weight.values[0]
            area=truck[truck["type"]==best].area.values[0]

            #refit route items into the best truck type
            x=[]  #for finding the largest length for each row
            row=1   #the row of crate
            df9["load"] = np.nan
            for i in range(len(df9)):
                if (df9.loc[i, 'Weight']<wt): #check weight constraint
                    if(df9.loc[i, 'positnum']==0): #packaged with other items, do not occupy space separately
                        df9.loc[i, 'load']="Yes" #marked as loaded
                        wt-=df9.loc[i, 'Weight'] #deduct left truck weight
                    else:
                        if (df9.loc[i, 'L']<l): #check length constraint
                            for j in range(int(df9.loc[i, 'positnum'])): #for each item that occupy 1 or more positions, check each position's constraints
                                if(df9.loc[i, 'W']<w): #check weight constraint
                                    w-=df9.loc[i, 'W'] #deduct left truck width                           
                                    x.append(df9.loc[i, 'L']) #record crates' length in the same row
                                    if j==int(df9.loc[i, 'positnum']-1): #check whether all the crates of the same item record have been loaded
                                        df9.loc[i, 'load']="Yes" #marked as loaded
                                        wt-=df9.loc[i, 'Weight'] #deduct left truck weight
                                        area-=df9.loc[i, 'positnum']*df9.loc[i, 'cratearea'] #deduct left truck area
                                    else:
                                        continue
                                else:
                                    l-=max(x) #deduct left truck length by the largest crate's length
                                    row+=1 #move to next row
                                    w=truck.loc[k,'width'] #reset the left truck width
                                    x=[] #release the record of crates' length in the same row
                                    if (df9.loc[i, 'L']<l):
                                        if(df9.loc[i, 'W']<w):
                                            w-=df9.loc[i, 'W']                            
                                            x.append(df9.loc[i, 'L'])
                                            if j==int(df9.loc[i, 'positnum']-1):
                                                df9.loc[i, 'load']="Yes"
                                                wt-=df9.loc[i, 'Weight']
                                                area-=df9.loc[i, 'positnum']*df9.loc[i, 'cratearea']
                                            else:
                                                continue
                        else:
                            break
                else:
                    break

            fullnum.append(df9["full"].sum())
            partnum.append((np.ceil(df9["part"])).sum())
            positnum.append(df9["positnum"].sum())
            unloadedfull.append(df9[df9["load"]!="Yes"]["full"].sum())
            unloadedpart.append(df9[df9["load"]!="Yes"]["part"].sum())
            leftwt.append(wt)  
            leftarea.append(area)

            if route==1:
                loaddetail=df9
            else:
                loaddetail=loaddetail.append(df9)
        loaddetail["load"].fillna("No", inplace=True)
        data1=pd.DataFrame({"RouteNo":routeno,"RouteName":routename,"RouteInfo":routeinfo,"Suggestion":routeresult,"Left Loading Capacity":leftwt,"FullCrateNum":fullnum,"PartialCrateNum":partnum,"PositionNum":positnum,"UnloadFullNum":unloadedfull,"UnloadPartNum":unloadedpart})
        data2=pd.DataFrame(loaddetail[["RouteNo","RouteName","Customer#","SO#","Item#","Product Line_x","Qty","Weight","cratenum","positnum","full","part","PcsPerCrate","WtPerCrate","length","width","L","W","CrateSizeHeight","maxstack","load"]])
        data2.rename(columns={"Product Line_x":"Product Line","length":"Originlength","width":"Originwidth","L":"CrateLength","W":"CrateWidth","CrateSizeHeight":"CrateHeight","cratenum":"CrateNum","maxstack":"MaxStackNum","positnum":"PositNum","full":"FullNum","part":"PartialNum"},inplace=True)
        data1.to_excel("Predict Result.xlsx", index=False)
        data2.to_excel("Predict Details.xlsx", index=False)
        sg.popup('Prediction Completed')
window.close()